In [9]:
import sys
import pandas as pd
import argparse
from tqdm import tqdm
import pysam
from collections import defaultdict
import scipy.sparse as sp
import anndata

## Testing computing readcounts and other stats with pysam

In [2]:
def count_barcodes(output_file=None):
    # Dictionaries for barcode counts and unique reads
    barcode_counts = defaultdict(int)
    unique_read_counts = defaultdict(set)

    # Dictionary to store total insert length per barcode
    insert_lengths = defaultdict(list)
    
    # Process reads with tqdm progress bar, without pre-counting lines
    with tqdm(desc="Computing reads per barcode", unit="reads") as pbar:
        for line in sys.stdin:
            # Split the line by tabs to parse the fields
            fields = line.strip().split()

            # Find the barcode (BC:Z:<barcode>)
            barcode = None
            for field in fields[11:]:  # SAM optional fields start at the 12th column
                if field.startswith("CB:Z:"):
                    barcode = field.split(":", 2)[2]  # Extract the barcode part
                    break

            if barcode:
                # Increment count for this barcode
                if barcode in barcode_counts:
                    barcode_counts[barcode] += 1
                else:
                    barcode_counts[barcode] = 1

            # Update progress bar after each read
            pbar.update(1)

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(list(barcode_counts.items()), columns=["barcode", "read_count"])

    # Output to CSV or standard output
    if output_file:
        df.to_csv(output_file, index=False)
        print(f"Saved barcode read counts to {output_file}")
    else:
        print(df.to_csv(index=False))

### Testing with pysam

In [3]:
def count_barcodes_with_insert_length(bam_file, output_file=None):
    # Dictionaries for barcode counts and unique reads
    barcode_counts = defaultdict(int)
    unique_read_counts = defaultdict(set)

    # Dictionary to store total insert length per barcode
    insert_lengths = defaultdict(list)

    # Open BAM file
    with pysam.AlignmentFile(bam_file, "rb") as bam:
        # Process reads with a progress bar
        with tqdm(desc="Processing BAM file", unit="reads") as pbar:
            for read in bam:
                if read.is_unmapped or read.mate_is_unmapped:
                    # Skip reads where either mate is unmapped
                    continue

                if not read.is_read1:
                    # Process only read1 for insert length calculation
                    continue

                # Parse the barcode
                cell_barcode = None
                for tag, value in read.tags:
                    if tag == "BC":
                        # Handle concatenated BC and CB
                        if "CB:Z:" in value:
                            parts = value.split(" CB:Z:")
                            cell_barcode = parts[1]
                        else:
                            continue
                    elif tag == "CB":
                        cell_barcode = value

                if not cell_barcode:
                    # Skip if no cell barcode is found
                    continue

                # Increment total read count for this barcode
                barcode_counts[cell_barcode] += 1

                # Compute insert length (requires paired-end reads)
                if read.is_proper_pair:
                    # Calculate 5' and 3' coordinates for insert length
                    five_prime = read.reference_start + 1  # 5' end (1-based)
                    three_prime = read.next_reference_start + read.template_length

                    insert_length = abs(three_prime - five_prime)

                    # Store insert length for the barcode
                    insert_lengths[cell_barcode].append(insert_length)

                # Update progress bar
                pbar.update(1)

    # Compute average insert length per barcode
    avg_insert_lengths = {
        barcode: sum(lengths) / len(lengths) if lengths else 0
        for barcode, lengths in insert_lengths.items()
    }

    # Create DataFrames for outputs
    barcode_df = pd.DataFrame(list(barcode_counts.items()), columns=["barcode", "read_count"])
    insert_length_df = pd.DataFrame(
        list(avg_insert_lengths.items()), columns=["barcode", "average_insert_length"]
    )

    # Merge both DataFrames on barcode
    result_df = pd.merge(barcode_df, insert_length_df, on="barcode", how="outer").fillna(0)

    # Output to CSV or standard output
    if output_file:
        result_df.to_csv(output_file, index=False)
        print(f"Saved barcode read counts and insert lengths to {output_file}")
    else:
        print(result_df.to_csv(index=False))

# Example usage (replace 'your_bam_file.bam' with the actual BAM file path):
# count_barcodes_with_insert_length('your_bam_file.bam')


In [4]:
bam_file='Barn10/Barn10_unfiltered.bam'
with pysam.AlignmentFile(bam_file, "rb") as bam:
    counter = 0
    barcode_counts = defaultdict(int)
    barcode_reads = defaultdict(list)
    insert_lengths = defaultdict(list)
    unmapped_counts = defaultdict(int)
    
    for read in bam:
        counter += 1


        if not read.is_read1:
            # Process only read1 for insert length calculation
            continue
        # print(read.tags)
        # Initialize barcode variables
        illumina_index = None
        cell_barcode = None

        # Loop through the tags
        for tag, value in read.tags:
            if tag == "BC":
                # Parse BC tag for potential concatenated CB value
                if "CB:Z:" in value:
                    parts = value.split(" CB:Z:")
                    illumina_index = parts[0]  # Everything before ' CB:Z:'
                    cell_barcode = parts[1]    # Everything after ' CB:Z:'
                else:
                    illumina_index = value
            elif tag == "CB":
                cell_barcode = value
        # print(illumina_index, cell_barcode)
        if not cell_barcode:
            # Skip if no cell barcode is found
            continue
        
        if read.is_unmapped or read.mate_is_unmapped:
            # Skip reads where either mate is unmapped
            unmapped_counts[cell_barcode] += 1
            continue

        barcode_counts[cell_barcode] += 1
        
        # Compute insert length (requires paired-end reads)
        if read.is_proper_pair:
            # Calculate 5' and 3' coordinates for insert length
            five_prime = read.reference_start + 1  # 5' end (1-based)
            three_prime = read.next_reference_start + read.template_length

            insert_length = abs(three_prime - five_prime)
            
            barcode_reads[cell_barcode].append((five_prime, three_prime))
            
            # print(five_prime, three_prime, insert_length)
            if insert_length < 1000:
                insert_lengths[cell_barcode].append(insert_length)
            
        if counter > 100000:
            break 

In [5]:

# Compute average insert length per barcode
avg_insert_lengths = {
    barcode: sum(lengths) / len(lengths) if lengths else 0
    for barcode, lengths in insert_lengths.items()
}

unique_read_counts = {barcode:len(set(barcode_reads[barcode])) for barcode in barcode_reads.keys()}


# Create DataFrames for outputs
barcode_counts_df = pd.DataFrame(list(barcode_counts.items()), columns=["barcode", "read_count"])
unmapped_counts_df = pd.DataFrame(list(unmapped_counts.items()), columns=["barcode", "unmapped_read_count"])

insert_length_df = pd.DataFrame(
    list(avg_insert_lengths.items()), columns=["barcode", "average_insert_length"]
)

unique_read_counts_df = pd.DataFrame(list(unique_read_counts.items()), columns=["barcode", "unique_read_count"])

# Merge both DataFrames on barcode
result_df = pd.merge(pd.merge(pd.merge(barcode_counts_df, unique_read_counts_df, on="barcode", how="outer"), insert_length_df, on="barcode", how="outer"), unmapped_counts_df,on="barcode", how="outer").fillna(0)
result_df

,barcode,read_count,unique_read_count,average_insert_length,unmapped_read_count
0,AATGGCAGAGGAAACGATGGACTCGCTGGATAAAGGTTGGATGCA,1.0,1.0,329.000000,0.0
1,AATGGCAGAGGAACCTTCAGACTCAGACGAACAAGGCGGTAAGTA,1.0,1.0,26.000000,0.0
2,AATGGCAGAGGAAGGCATTGACTCTCCAGTCTAAGGGTCCGATTA,1.0,1.0,235.000000,0.0
3,AATGGCAGAGGAATGGTGTGACTCACAGTAGCAAGGGTTACGGTA,988.0,771.0,230.702306,1.0
4,AATGGCAGAGGAATGGTGTGACTCAGAATGCCAAGGTGACAGTGA,34.0,25.0,234.218750,0.0
...,...,...,...,...,...
299,TGTGGTTGAGGAGATACCGAACTCCAGGTGAAAAGGCATACCGTA,1.0,1.0,148.000000,0.0
300,TGTGGTTGAGGAGATACCGAACTCCATCGTTGAAGGTTGATGGCA,1.0,1.0,141.000000,0.0
301,TGTGGTTGAGGAGGAACCAAACTCTCCAGTCTAAGGTTGATGGCA,481.0,356.0,233.849462,1.0
302,TGTGGTTGAGGAGGACGAATACTCAGCATTGGAAGGGTCGGTAAA,227.0,173.0,263.654545,0.0


In [8]:
result_df.iloc[0]['barcode']

'AATGGCAGAGGAAACGATGGACTCGCTGGATAAAGGTTGGATGCA'

## Check gDNA inserts in empty SPCs

In [14]:
bam_file='Barn10/Barn10_unfiltered.bam'
barcode_reads = defaultdict(list)
with pysam.AlignmentFile(bam_file, "rb") as bam:
    counter = 0
    
    for read in bam:
        counter += 1


        if not read.is_read1:
            continue
            
        if read.is_unmapped or read.mate_is_unmapped:
            # Skip reads where either mate is unmapped
            continue

        cell_barcode = None

        # Loop through the tags
        for tag, value in read.tags:
            if tag == "BC":
                # Parse BC tag for potential concatenated CB value
                if "CB:Z:" in value:
                    parts = value.split(" CB:Z:")
                    illumina_index = parts[0]  # Everything before ' CB:Z:'
                    cell_barcode = parts[1]    # Everything after ' CB:Z:'
                else:
                    illumina_index = value
            elif tag == "CB":
                cell_barcode = value
                
        if not cell_barcode:
            # Skip if no cell barcode is found
            continue
        
        # Compute insert length (requires paired-end reads)
        if read.is_proper_pair:
            # Calculate 5' and 3' coordinates for insert length
            chrom = read.reference_name
            five_prime = read.reference_start + 1  # 5' end (1-based)
            three_prime = read.next_reference_start + read.template_length

            insert = f'{chrom}:{five_prime}-{three_prime}'
            insert_length = three_prime - five_prime
            if insert_length < 1000:
                barcode_reads[cell_barcode].append(insert)
            
        if counter > 100000:
            break 
# barcode_reads

In [16]:
def create_count_matrix(barcode_reads):
    # Step 1: Identify all unique gDNA inserts and barcodes
    all_inserts = set()
    all_barcodes = set()

    # Count occurrences of each (barcode, insert) pair
    counts = defaultdict(int)

    for barcode, inserts in barcode_reads.items():
        all_barcodes.add(barcode)
        for insert in inserts:
            all_inserts.add(insert)
            counts[(barcode, insert)] += 1

    # Sort barcodes and inserts for consistent ordering
    all_inserts = sorted(all_inserts)
    all_barcodes = sorted(all_barcodes)

    # Step 2: Map barcodes and inserts to matrix indices
    insert_idx = {insert: idx for idx, insert in enumerate(all_inserts)}
    barcode_idx = {barcode: idx for idx, barcode in enumerate(all_barcodes)}

    # Step 3: Populate the sparse matrix
    row_indices = []
    col_indices = []
    values = []

    for (barcode, insert), count in counts.items():
        row_indices.append(barcode_idx[barcode])
        col_indices.append(insert_idx[insert])
        values.append(count)  # Use the count of occurrences

    count_matrix = sp.csr_matrix(
        (values, (row_indices, col_indices)),
        shape=(len(all_barcodes), len(all_inserts)),
    )

    # Step 4: Create AnnData object
    adata = anndata.AnnData(
        X=count_matrix,
        obs=pd.DataFrame(index=all_barcodes),
        var=pd.DataFrame(index=all_inserts),
    )

    return adata

In [22]:
adata = create_count_matrix(barcode_reads)
adata

AnnData object with n_obs × n_vars = 283 × 37410

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 37431 stored elements and shape (283, 37410)>